# Introduction to atomman: table conversions

__Lucas M. Hale__, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), _Materials Science and Engineering Division, NIST_.
    
[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 

## 1. Introduction<a id='section1'></a>

Atomic configuration data is often expressed in a tabular form as it is an efficient way of representing per-atom properties.  Many commn formats, such as the xyz format, are little more than this.  In atomman, the table load/dump style provides a means of converting to/from a generic table format.  As such, the table-style converter can be used to help with defining converters from more specific tabular formats.

**Library Imports**

In [1]:
# Standard Python libraries
import datetime

# http://www.numpy.org/
import numpy as np

import atomman as am
import atomman.unitconvert as uc

# Show atomman version
print('atomman version =', am.__version__)

# Show date of Notebook execution
print('Notebook executed on', datetime.date.today())

atomman version = 1.4.10
Notebook executed on 2023-07-28


Generate test system information (CsCl)

In [2]:
# Generate box
alat = uc.set_in_units(3.2, 'angstrom')
box = am.Box(a=alat, b=alat, c=alat)

# Generate atoms with atype, pos, charge, and stress properties
atype = [1, 2]
pos = [[0,0,0], [0.5, 0.5, 0.5]]
charge = uc.set_in_units([1, -1], 'e')
stress = uc.set_in_units(np.zeros((2, 3, 3)), 'MPa')
atoms = am.Atoms(pos=pos, atype=atype, charge=charge, stress=stress)

# Build system from box and atoms, and scale atoms
system = am.System(atoms=atoms, box=box, scale=True, symbols=['Cs', 'Cl'])

# Print system information
print(system)
system.atoms_df()

avect =  [ 3.200,  0.000,  0.000]
bvect =  [ 0.000,  3.200,  0.000]
cvect =  [ 0.000,  0.000,  3.200]
origin = [ 0.000,  0.000,  0.000]
natoms = 2
natypes = 2
symbols = ('Cs', 'Cl')
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos', 'charge', 'stress']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       2 |   1.600 |   1.600 |   1.600


,atype,pos[0],pos[1],pos[2],charge,stress[0][0],stress[0][1],stress[0][2],stress[1][0],stress[1][1],stress[1][2],stress[2][0],stress[2][1],stress[2][2]
0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1.6,1.6,1.6,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2. System.dump('table')<a id='section2'></a>

Parameters

- **f** (*str or file-like object, optional*) File path or file-like object to write the table to.  If not given, then the table is returned as a string.

- **prop_name** (*list, optional*) The Atoms properties to include.  Must be given if prop_info is not.

- **table_name** (*list, optional*) The table column name(s) that correspond to each prop_name.  If not given, the table_name values will be based on the prop_name values.

- **shape** (*list, optional*) The shape of each per-atom property.  If not given, will be inferred from the length of each table_name value.

- **unit** (*list, optional*) Lists the units for each prop_name as stored in the table.  For a value of None, no conversion will be performed for that property.  For a value of 'scaled', the corresponding table values will be taken in box-scaled units.  If not given, all unit values will be set to None (i.e. no conversions).

- **dtype** (*list, optional*) Allows for the data type of each property to be explicitly given. Values of None will infer the data type from the corresponding property values.  If not given, all values will be None.

- **prop_info** (*list of dict, optional*) Structured form of property conversion parameters, in which each dictionary in the list corresponds to a single atoms property.  Each dictionary must have a 'prop_name' field, and can optionally have 'table_name', 'shape', 'unit', and 'dtype' fields.

- **header** (*bool, optional*) Flag indicating whether to include the column names in the outputted table.  Default value is False (no column names).

- **float_format** (*str, optional*) c-style formatting string for floating point values.  Default value is '%.13f'.

- **return_prop_info** (*bool, optional*) Flag indicating if the filled-in prop_info is to be returned.  Having this allows for 1:1 load/dump conversions.  Default value is False (prop_info is not returned).

- **extra** (*dict, optional*)  Allows extra per-atom data that is not part of the System to be included in the generated table.  Useful when the per-atom data only has meaning in the tabular format and should not be added to System.

Returns

- (*str*) The generated data table.  Only returned if fp is None.

Dump data without a header

In [3]:
table = system.dump('table', float_format='%.5f')
print(table)

1 0.00000 0.00000 0.00000 1.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000
2 1.60000 1.60000 1.60000 -1.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000



Dump data with a header

In [4]:
table = system.dump('table', float_format='%.5f', header=True)
print(table)

atype pos[0] pos[1] pos[2] charge stress[0][0] stress[0][1] stress[0][2] stress[1][0] stress[1][1] stress[1][2] stress[2][0] stress[2][1] stress[2][2]
1 0.00000 0.00000 0.00000 1.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000
2 1.60000 1.60000 1.60000 -1.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000



Setting return_prop_info=True will also return a list of dictionaries that provides the transformation mapping between the table format and the scalar/vector/tensor per-atom properties.

In [5]:
table, prop_info = system.dump('table', float_format='%.5f', header=True, return_prop_info=True)
print(table)

atype pos[0] pos[1] pos[2] charge stress[0][0] stress[0][1] stress[0][2] stress[1][0] stress[1][1] stress[1][2] stress[2][0] stress[2][1] stress[2][2]
1 0.00000 0.00000 0.00000 1.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000
2 1.60000 1.60000 1.60000 -1.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000



In [6]:
# Show the generated prop_info conversion info
for pinfo in prop_info:
    print(pinfo)

{'prop_name': 'atype', 'shape': (), 'table_name': ['atype'], 'unit': None, 'dtype': None}
{'prop_name': 'pos', 'shape': (3,), 'table_name': ['pos[0]', 'pos[1]', 'pos[2]'], 'unit': None, 'dtype': None}
{'prop_name': 'charge', 'shape': (), 'table_name': ['charge'], 'unit': None, 'dtype': None}
{'prop_name': 'stress', 'shape': (3, 3), 'table_name': ['stress[0][0]', 'stress[0][1]', 'stress[0][2]', 'stress[1][0]', 'stress[1][1]', 'stress[1][2]', 'stress[2][0]', 'stress[2][1]', 'stress[2][2]'], 'unit': None, 'dtype': None}


## 3. atomman.load('table')<a id='section3'></a>

Parameters

- **table** (*str or file-like object*) The table content, file path or file-like object containing the content to read.

- **box** (*atomman.Box*) The atomic box to use when generating a System around the data.

- **symbols** (*tuple, optional*) Allows the list of element symbols to be assigned during loading.

- **system** (*atomman.System, optional*) The atomic system to load the values to.  If not given, a new system will be constructed.

- **prop_name** (*list, optional*) The Atoms properties to generate.  Must be given if prop_info is not.

- **table_name** (*list, optional*) The table column name(s) that correspond to each prop_name.  If not given, the table_name values will be based on the prop_name values.

- **shape** (*list, optional*) The shape of each per-atom property.  If not given, will be inferred from the length of each table_name value.

- **unit** (*list, optional*) Lists the units for each prop_name as stored in the table.  For a value of None, no conversion will be performed for that property.  For a value of 'scaled', the corresponding table values will be taken in box-scaled units.  If not given, all unit values will be set to None (i.e. no conversions).

- **dtype** (*list, optional*) Allows for the data type of each property to be explicitly given.  Values of None will infer the data type from the corresponding property values.  If not given, all values will be None.

- **prop_info** (*list of dict, optional*) Structured form of property conversion parameters, in which each dictionary in the list corresponds to a single atoms property.  Each dictionary must have a 'prop_name' field, and can optionally have 'table_name', 'shape', 'unit', and 'dtype' fields.

- **skiprows** (*int, optional*) Number of rows to skip before reading the data.

- **usecols** (*int, optional*) Which columns are to be used. Will be passed to pandas.read_csv() usecols option.

- **nrows** (*int, optional*) Number of rows of data to read.

- **comment** (*str, optional*) Specifies a character which indicates all text on a given line after is to be considered to be a comment and ignored by parser. This is often '#'.

Returns

- (*atomman.System*) The generated system.

atomman.load('table') has a large number of parameters to allow many varieties of tabular data to be interpreted.  Here, parameters are selected to read in the table generated above to reproduce the original system.

In [7]:
# Show table and prop_info again
print(table)
print()
print(prop_info)

atype pos[0] pos[1] pos[2] charge stress[0][0] stress[0][1] stress[0][2] stress[1][0] stress[1][1] stress[1][2] stress[2][0] stress[2][1] stress[2][2]
1 0.00000 0.00000 0.00000 1.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000
2 1.60000 1.60000 1.60000 -1.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000


[{'prop_name': 'atype', 'shape': (), 'table_name': ['atype'], 'unit': None, 'dtype': None}, {'prop_name': 'pos', 'shape': (3,), 'table_name': ['pos[0]', 'pos[1]', 'pos[2]'], 'unit': None, 'dtype': None}, {'prop_name': 'charge', 'shape': (), 'table_name': ['charge'], 'unit': None, 'dtype': None}, {'prop_name': 'stress', 'shape': (3, 3), 'table_name': ['stress[0][0]', 'stress[0][1]', 'stress[0][2]', 'stress[1][0]', 'stress[1][1]', 'stress[1][2]', 'stress[2][0]', 'stress[2][1]', 'stress[2][2]'], 'unit': None, 'dtype': None}]


In [8]:
# Pass table with box and prop_info
table_system = am.load('table', table, box=system.box, symbols=['Cs', 'Cl'],
                       skiprows=1, prop_info=prop_info)

print(table_system)
table_system.atoms_df()

avect =  [ 3.200,  0.000,  0.000]
bvect =  [ 0.000,  3.200,  0.000]
cvect =  [ 0.000,  0.000,  3.200]
origin = [ 0.000,  0.000,  0.000]
natoms = 2
natypes = 2
symbols = ('Cs', 'Cl')
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos', 'charge', 'stress']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       2 |   1.600 |   1.600 |   1.600


,atype,pos[0],pos[1],pos[2],charge,stress[0][0],stress[0][1],stress[0][2],stress[1][0],stress[1][1],stress[1][2],stress[2][0],stress[2][1],stress[2][2]
0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1.6,1.6,1.6,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Using the generated prop_info from dump makes the resulting properties have the right shape.
print(table_system.atoms.stress[0])

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
